<a href="https://colab.research.google.com/github/veraDanilova/COV19_user-topic_networks/blob/main/GraphType2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

!pip install pyLDAvis
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install -U pandas-profiling
!pip install dash-table
!pip install pyngrok==4.1.1


In [2]:
!ngrok authtoken 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!git clone https://github.com/veraDanilova/COV19_user-topic_networks.git

COPY_FILE = True
! unzip /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv.zip -d /content/COV19_user-topic_networks/Proj_Data/
! unzip /content/COV19_user-topic_networks/Proj_Data/COR_TWI_resized.csv.zip -d /content/COV19_user-topic_networks/Proj_Data/

fatal: destination path 'COV19_user-topic_networks' already exists and is not an empty directory.
Archive:  /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv.zip
replace /content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# PANDAS TEST. В случае ошибки перезапустить среду выполнения

import pandas as pd
DF = pd.read_csv('/content/COV19_user-topic_networks/Proj_Data/COR_LJ_resized.csv', index_col = 0, engine = 'python')
DF.head()

,index,tag,linkTo_post,title,acc_name,tags,date,name,birthdate,location,fulltext,sents,unigramsC,text_length
0,13802.0,коронавирус,https://igorek3d.livejournal.com/743845.html,(Без заголовка),igorek3d,['коронавирус'],2020-03-22 17:40:01,Иванов Игорь Анатольевич,None,"{'country-name': 'Russian Federation', 'region...","хз, у нас полная тишина, вчера был в тренажерн...","['хз, у нас полная тишина, вчера был в тренаже...","['полный', 'тишина', 'вчера', 'тренажерный', '...",33.0
1,13800.0,коронавирус,https://grimnir74.livejournal.com/12515729.html,Министерство здравоохранения одобрило ... набо...,grimnir74,['коронавирус'],2020-03-22 17:45:09,Алексей С. Железнов,26 августа 1974,"{'country-name': 'Ukraine', 'locality': 'Никол...",Министерство здравоохранения разрешило выпуск ...,['Министерство здравоохранения разрешило выпус...,"['министерство', 'здравоохранение', 'разрешать...",153.0
2,13799.0,коронавирус,https://geomakss.livejournal.com/1724275.html,КПРФ следует наказать за митинг во время эпидемии,geomakss,"['кпрф', 'коммунизм', 'политика', 'коронавирус...",2020-03-22 17:46:36,geomakss,20 августа,"{'country-name': 'Russian Federation', 'region...",Похоже коммунисты вообразили себя непробиваемы...,['Похоже коммунисты вообразили себя непробивае...,"['похоже', 'коммунист', 'воображать', 'непроби...",116.0
3,13798.0,коронавирус,https://umgezogen.livejournal.com/147000.html,"Очень рекомендую посмотреть, как там, в Китае",umgezogen,"['коронавирус', 'правительство скрывает', 'кет...",2020-03-22 17:49:34,umgezogen,14 августа 1967,"{'country-name': 'Germany', 'locality': 'Nürnb...","Чтобы вы понимали 1. Кого там лечат, сколько э...","['Чтобы вы понимали 1.', 'Кого там лечат, скол...","['понимать', 'лечить', 'стоять', 'доступный', ...",15.0
4,13797.0,коронавирус,https://denyaleto.livejournal.com/206525.html,"Дончанин в Шэньчжэнь: ""Китай находится на фини...",denyaleto,"['аваков', 'донецк', 'covid-19', 'шэньчжэнь', ...",2020-03-22 17:51:03,denyaleto,28 декабря,"{'country-name': 'Ukraine', 'locality': 'Донецк'}",В бытность моего студенчества многие мои сокур...,['В бытность моего студенчества многие мои сок...,"['бытность', 'студенчество', 'сокурсник', 'гов...",901.0


# GRAPH TYPE 2 JupyterDash

"Add folder to My Drive". It will create a symlink (it won't copy, won't take space). He can now mount it the same way as you, using drive.mount(..).

Share the folder with him. Then access the folder and files within using pydrive, not drive.mount(). You must use the FILE_ID instead of file names to refer to those folder and files.




In [10]:
# %%writefile COV19_user-topic_networks.py
import sys
sys.path.append('/content/COV19_user-topic_networks/Proj_Data')
"""
BIDIRECTED GRAPH, UNIFIED TOPIC TABLE

"""

# general
import re
import string
import gzip
import os
import itertools
import sklearn.preprocessing

# data
import pandas as pd
import json
import gensim
import gensim.corpora as corpora


# LDA
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet
import pyLDAvis

# Plotting tools
import matplotlib.pyplot as plt
# %matplotlib inline
import networkx as nx
import plotly.io as pio
pio.renderers.default='notebook'
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)


from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import LoadGraphData
from LoadGraphData import GraphType2D
proxy = JupyterDash.infer_jupyter_proxy_config()

root_folder = '/content/COV19_user-topic_networks/Proj_Data/'

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])

wnums = ['w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10']
week_dates = [("2020-03-22 - 2020-03-29"), 
        ("2020-03-29 - 2020-04-05"),
        ("2020-04-05 - 2020-04-12"),
        ("2020-04-12 - 2020-04-19"),
        ("2020-04-19 - 2020-04-26"),
        ("2020-04-26 - 2020-05-03"),
        ("2020-05-03 - 2020-05-10"),
        ("2020-05-10 - 2020-05-17"),
        ("2020-05-17 - 2020-05-24"),
        ("2020-05-24 - 2020-06-01")]
app.layout = html.Div([
        html.H1("Weekly changes of User Attention to COVID-related topics (March 22 - June 1, 2020)", style = {'color': 'navy'}),
        html.Div(
            [html.Div([
            dcc.Dropdown(
                id='week-filter',
                options=[{'label': i +' : '+ wd, 'value': i} for i,wd in zip(wnums, week_dates)],
                value='w1'
            )], style={'width': '50%'}),
            
            html.Div([
            dcc.RadioItems(
                id='cluster-filter',
                options=[{'label': 'Cluster of '+ t, 'value': i} for i,t in zip([0, 1],['uniform activity','one-week users'])],
                value=0,
                labelStyle={'display': 'inline-block'},
                # className = 'button'           
            ),
            
            dcc.RadioItems(
                id='net-filter',
                options=[{'label': i, 'value': i} for i in ['LJ', 'Twi']],
                value='LJ',
                labelStyle={'display': 'inline-block'},
                # className = 'button'
                )
        ])
            ],
        style={'display': 'flex'}),
    
        html.Div([
            dcc.Graph(id = 'dd-output-container')],
        style={'width': '70%'})
            ])
 

@app.callback(
    dash.dependencies.Output('dd-output-container', 'figure'),
#     [dash.dependencies.Input('week'+i, 'value') for i in wnums]+
    [dash.dependencies.Input('week-filter', 'value'),
    dash.dependencies.Input('cluster-filter', 'value'),
    dash.dependencies.Input('net-filter', 'value')]
)
def update_output(weekfilter, clusterfilter, netfilter):
    
    net = netfilter
    wnum = weekfilter
    cl_id = clusterfilter

    G = GraphType2D(net,wnum,cl_id,root_folder)
    fig = G.runBuildGraph()

    return fig
# if __name__ == '__main__':
    # app.run_server(mode='external', port = '8050',debug = True)
    # app.run_server(host='127.0.0.1', port='8050', proxy=proxy, debug=False, dev_tools_ui=None, dev_tools_props_check=None, dev_tools_serve_dev_bundles=None, dev_tools_hot_reload=None, dev_tools_hot_reload_interval=None, dev_tools_hot_reload_watch_interval=None, dev_tools_hot_reload_max_retry=None, dev_tools_silence_routes_logging=None, dev_tools_prune_errors=None)
# app.run_server(host='127.0.0.1', port='8050')
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

In [9]:
# # Open a ngrok tunnel to the HTTP server
# public_url = ngrok.connect(5000).public_url
# print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# # Update any base URLs to use the public ngrok URL
# app.config["BASE_URL"] = public_url


from pyngrok import ngrok
# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '8050')


t=2021-04-24T12:19:45+0000 lvl=warn msg="failed to start tunnel" pg=/api/tunnels id=1bb82dbe29352e34 err="Your account may not run more than 4 tunnels over a single ngrok client session.\nThe tunnels already running on this session are:\n[]\n\r\n\r\nERR_NGROK_324\r\n"



PyngrokNgrokHTTPError: ignored

In [8]:
# public_url
ngrok.disconnect(public_url)

In [ ]:
! python COV19_user-topic_networks.py s

/usr/local/lib/python3.7/dist-packages/past/translation/__init__.py:35: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/py

In [ ]:
app._terminate_server_for_port("localhost", 8050)